In [1]:
import plotly.plotly as py
import ipywidgets as widgets
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import plotly.graph_objs as go

from ipywidgets import interact, interactive, fixed
from IPython.core.display import HTML
from IPython.display import display, clear_output
from plotly.widgets import GraphWidget
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

<IPython.core.display.Javascript object>

In [2]:
init_notebook_mode(connected=True)

In [3]:
train_new = pd.read_feather("data/xc_train.feather")

In [4]:
valid_new = pd.read_feather("data/xc_valid.feather")

In [5]:
rf = joblib.load('./data/rfc_trained.pkl')

In [6]:
y_train = pd.read_csv("data/yc_train.csv")

In [7]:
y_valid = pd.read_csv("data/yc_valid.csv")

In [8]:
valid_preds = rf.predict_proba(valid_new)[:,1]

In [9]:
train_preds = rf.predict_proba(train_new)[:,1]

# Add slider

In [10]:
step_size =[round(0.01*i,2) for i in range(101)]

In [11]:
def hard_preds(train, valid, value):
    train2 = train.copy()
    train2[train2<value] = 0
    train2[train2>value] = 1
    valid2 = valid.copy()
    valid2[valid2<value]=0
    valid2[valid2>value] =1
    return np.array([int(sum(train2)), int(len(train2) - sum(train2)), 
                     int(sum(valid2)), int(len(valid2) - sum(valid2))])

In [12]:
data = [dict(
        visible = False,
        name = '𝜈 = '+str(step),
        mode = 'markers',
        x = np.array([0,1, 0, 1]),
        y = hard_preds(train_preds, valid_preds, step)) for step in step_size]
        #x= x_ys[0],
        #y = x_ys[1]
data[10]['visible'] = True

In [13]:
steps = []
for i in range(len(data)):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data)],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 10,
    currentvalue = {"prefix": "Frequency: "},
    pad = {"t": 50},
    steps = steps
)]

In [14]:
layout = dict(sliders=sliders)
layout.update(yaxis=dict(
        autorange = True,
        #range = [0,max_y],
        tick0=0),
              title='Number of Hard Predictions for Training and Validation'
             )

#fig = dict([go.Histogram(x=data)])
#data = [go.Histogram(data["bins"])]

#fig = go.Histogram(y=data, layout=layout)
fig = dict(data=data, layout=layout)


iplot(fig, filename='Sine Wave Slider')